In [2]:
import os
os.chdir('applications/vimc_example')

import sys
sys.path.append('applications/vimc_example')

import pandas as pd
import numpy as np
import atomica as at

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'applications/vimc_example'

In [4]:
import vimc_functions as vimc
import hbv_functions as hbv

In [6]:
os.chdir('C:/Users/Phil/Documents/GitHub/hbv-globalinvcase')

In [8]:
regions = ['AFR', 'AMR', 'EMR', 'EUR', 'SEAR', 'WPR']
fw_loc = 'frameworks/hbv_v15_csdev_b.xlsx'

F = at.ProjectFramework(fw_loc)
D = at.ProjectData.from_spreadsheet(f"applications/vimc_WPR_test/WPR_db.xlsx", framework=F)
P = at.Project(framework=F, databook=D, sim_dt=0.25, sim_start=1990, sim_end=2099, do_run=False)
cal = P.make_parset()
cal.load_calibration(f"applications/vimc_WPR_test/WPR_test_calib.xlsx")
res = P.run_sim(parset=cal)

Parameter "m_dc" is in rate units and a maximum value of "1" has been entered. Rates in the framework should generally not be limited to "1"
Parameter "m_hcc" is in rate units and a maximum value of "1" has been entered. Rates in the framework should generally not be limited to "1"


Elapsed time for running "default": 6.08s


In [11]:
from hbv_functions import metric

In [9]:
ct = 'WPR'

In [12]:
edf = pd.DataFrame()
years = [2000,2010, 2015, 2019]
pars = ['chb_pop', 'cl_acu', 'cl_cir', 'cl_hcc']
for par in pars:
    data = []
    est = []

    if par == 'chb_pop':
        tmp_yr = years[1:]
    else:
        tmp_yr = years

    for yr in tmp_yr:
        tmp = [P.data.tdve[par].ts[x].interpolate(2000 + yr) for x in range(len(D.pops))]  # data
        data = [i[0] for i in tmp]
        est = [x.vals[(yr - 2000) * 4] for x in res.get_variable(par)]  # estimate

        edf.loc[f"{ct}", par] = metric(data, est, "mape")

edf = edf.apply(pd.to_numeric).round(4)

In [13]:
edf

,chb_pop,cl_acu,cl_cir,cl_hcc
WPR,0.3572,0.3689,1.2752,0.9872


In [14]:
hbv.determine_cal_quality(edf)

,chb_pop,cl_acu,cl_cir,cl_hcc
WPR,0.357200,0.368900,1.275200,0.987200


### Functions

In [ ]:
def error_table(regions, fw_loc='frameworks/hbv_v14_gamma_mav.xlsx'):
    '''
    THIS ONLY WORKS WITH WHO REGIONS RIGHT NOW
    TODO: comment on how parts of the funtion work
    '''
    edf = pd.DataFrame()
    years = [2010, 2015, 2019]
    pars = ['chb_pop', 'cl_acu', 'cl_cir', 'cl_hcc']
    F = at.ProjectFramework(fw_loc)
    for ct in regions:
        print(f"Current region: {ct}")

        D = at.ProjectData.from_spreadsheet(f"applications/region_{ct.lower()}/{ct}_db_mav.xlsx", framework=F)
        P = at.Project(framework=F, databook=D, sim_dt=0.25, sim_start=1990, sim_end=2099, do_run=False)
        cal = P.make_parset()
        cal.load_calibration(f"applications/region_{ct.lower()}/{ct}_calib.xlsx")
        res = P.run_sim(parset=cal)

        for par in pars:
            data = []
            est = []

            if par == 'chb_pop':
                tmp_yr = years[1:]
            else:
                tmp_yr = years

            for yr in tmp_yr:
                tmp = [P.data.tdve[par].ts[x].interpolate(2000 + yr) for x in range(len(D.pops))]  # data
                data = [i[0] for i in tmp]
                est = [x.vals[(yr - 2000) * 4] for x in res.get_variable(par)]  # estimate

                edf.loc[f"{ct}", par] = metric(data, est, "mape")

    edf = edf.apply(pd.to_numeric).round(4)
    return edf

In [ ]:
def central_results(fw = 'hbv_v14_gamma_2.xlsx', db = "AFR_db_v1_2_1.xlsx", cl = 'AFR_calib_v1_2.xlsx'):
    '''
    Created by: Phillip Luong (https://github.com/phillipluong)
    Last Updated: 31/01/23
    Generates a DataFrame listing all central output results.

    Inputs:
    - fw:       Excel sheet of the Atomica Framework of the model (str)
    - db:       Excel sheet of the Atomica Databook of the model (str)
    - cl:       Excel sheet of the Model Calibration of the model - specific to a region estimate (str)

    Outputs:
    - cen_df:   DataFrame listing the central estimates of 1-year age groups between 2000-2100 (DataFrame)
    '''
    P, res = load_cen_project(fw= fw, db=db, cl=cl)
    ## Locations for the important dataframes
    loc = 'applications/vimc/Data/Templates/' # The location for the output templates may be different
    loc2 = 'applications/vimc/Data/Demographics/' # The location for the input data may be different (depending on where you store it)

    ## Load and process the input data
    df1 = pd.read_csv(loc2+'202212rfp-1_dds-202208_int_pop_both.csv')

    df1 = df1[(df1.year >= 1990) & (df1.year <=2100)]
    age_groups = ['0-4', '5-14', '15-49', '50-69', '70+']

    ## Assort data by age group
    for idx,row in df1.iterrows():
        if row.age_from < 5:
            df1.loc[idx,'age_group'] = '0-4'
        elif row.age_from < 15:
            df1.loc[idx,'age_group'] = '5-14'
        elif row.age_from < 50:
            df1.loc[idx,'age_group'] = '15-49'
        elif row.age_from < 70:
            df1.loc[idx,'age_group'] = '50-69'
        else:
            df1.loc[idx,'age_group'] = '70+'


    ## Start generating output results
    output_dict = {'cohort_size': 'alive', 'cases': 'tot_inc', 'dalys':'dalys'}
    cen_df = pd.read_csv(loc+'/central-burden-template.202212rfp-1.RFP_standard template_HepB.csv')

    for opt in output_dict:
        tot_val = res.get_variable(output_dict[opt])[0].vals + res.get_variable(output_dict[opt])[1].vals
        for age in range(0,5):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value'] # Ratio of age relative to age group
                res_val = tot_val[(year-1990)*4] # The '4' should be timestep (if future runs are different)

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0] #Index of age group year sim entry
                cen_df.loc[idx, opt] = ratio * res_val

        tot_val = res.get_variable(output_dict[opt])[2].vals + res.get_variable(output_dict[opt])[3].vals
        for age in range(5,15):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
                cen_df.loc[idx, opt] = ratio * res_val

        tot_val = res.get_variable(output_dict[opt])[4].vals + res.get_variable(output_dict[opt])[5].vals
        for age in range(15,50):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
                cen_df.loc[idx, opt] = ratio * res_val

        tot_val = res.get_variable(output_dict[opt])[6].vals + res.get_variable(output_dict[opt])[7].vals
        for age in range(50,70):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
                cen_df.loc[idx, opt] = ratio * res_val

        tot_val = res.get_variable(output_dict[opt])[8].vals + res.get_variable(output_dict[opt])[9].vals
        for age in range(70,101):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
                cen_df.loc[idx, opt] = ratio * res_val

    ## Deaths are calculated separately
    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[0].vals + res.get_variable(var)[1].vals # Total deaths in age group

    for age in range(0,5):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[2].vals + res.get_variable(var)[3].vals
    for age in range(5,15):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[4].vals + res.get_variable(var)[5].vals
    for age in range(15,50):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[6].vals + res.get_variable(var)[7].vals
    for age in range(50,70):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[8].vals + res.get_variable(var)[9].vals
    for age in range(70,101):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    return cen_df

In [ ]:
def stochastic_results(n_samples, fw = 'hbv_v14_gamma_2.xlsx', db = "AFR_db_v1_2_1.xlsx", cl = 'AFR_calib_v1_2.xlsx', seed = 310123):
    '''
    Created by: Phillip Luong (https://github.com/phillipluong)
    Last Updated: 31/01/23
    Generates a DataFrame listing all stochastic output results. Each stochastic output is indexed by a 'run_id' which can also refer to the stochastic input parameter set.

    Inputs:
    - n_samples:    Number of stochastic input samples (int)
    - fw:           Excel sheet of the Atomica Framework of the model (str)
    - db:           Excel sheet of the Atomica Databook of the model (str)
    - cl:           Excel sheet of the Model Calibration of the model - specific to a region estimate (str)
    - seed:         Random seed (int)

    Outputs:
    - final_df:     DataFrame listing all stochastic estimates of 1-year age groups between 2000-2100 (DataFrame)
    '''
    P, parsets, results = load_sto_project(n_samples, fw= fw, db=db, cl=cl, seed=seed)
    ## Locations for the important dataframes
    loc = 'Data/Templates/' # The location for the output templates may be different
    loc2 = 'Data/Demographics/' # The location for the input data may be different (depending on where you store it)

    df = pd.read_csv(loc+'/stochastic-burden-template.202212rfp-1.RFP_standard template_HepB.csv') #template

     ## Load and process the input data
    df1 = pd.read_csv(loc2+'202212rfp-1_dds-202208_int_pop_both.csv')

    df1 = df1[(df1.year >= 1990) & (df1.year <=2100)]
    age_groups = ['0-4', '5-14', '15-49', '50-69', '70+']

    for idx,row in df1.iterrows():
        if row.age_from < 5:
            df1.loc[idx,'age_group'] = '0-4'
        elif row.age_from < 15:
            df1.loc[idx,'age_group'] = '5-14'
        elif row.age_from < 50:
            df1.loc[idx,'age_group'] = '15-49'
        elif row.age_from < 70:
            df1.loc[idx,'age_group'] = '50-69'
        else:
            df1.loc[idx,'age_group'] = '70+'

    dfs = {}
    output_dict = {'cohort_size': 'alive', 'cases': 'tot_inc', 'dalys':'dalys'}
    for sim in range(1,n_sa                                      mples+1):
        stores = results[sim]
        print(f'Sim no {sim}')

        dfs[sim] = pd.read_csv(loc+'/stochastic-burden-template.202212rfp-1.RFP_standard template_HepB.csv')
        dfs[sim].run_id = sim
        ## Other outputs
        for opt in output_dict:
            print(opt)
            tot_val = stores.get_variable(output_dict[opt])[0].vals + stores.get_variable(output_dict[opt])[1].vals
            for age in range(0,5):
                for year in range(2000,2101):
                    ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value'] # Ratio of age relative to age group
                    res_val = tot_val[(year-1990)*4] # '4' refers to the timestep, 1990 refers to the beginning of the simulation

                    idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                    dfs[sim].loc[idx, opt] = ratio * res_val

            tot_val = stores.get_variable(output_dict[opt])[2].vals + stores.get_variable(output_dict[opt])[3].vals
            for age in range(5,15):
                for year in range(2000,2101):
                    ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
                    res_val = tot_val[(year-1990)*4]

                    idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                    dfs[sim].loc[idx, opt] = ratio * res_val

            tot_val = stores.get_variable(output_dict[opt])[4].vals + stores.get_variable(output_dict[opt])[5].vals
            for age in range(15,50):
                for year in range(2000,2101):
                    ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
                    res_val = tot_val[(year-1990)*4]

                    idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                    dfs[sim].loc[idx, opt] = ratio * res_val

            tot_val = stores.get_variable(output_dict[opt])[6].vals + stores.get_variable(output_dict[opt])[7].vals
            for age in range(50,70):
                for year in range(2000,2101):
                    ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
                    res_val = tot_val[(year-1990)*4]

                    idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                    dfs[sim].loc[idx, opt] = ratio * res_val

            tot_val = stores.get_variable(output_dict[opt])[8].vals + stores.get_variable(output_dict[opt])[9].vals
            for age in range(70,101):
                for year in range(2000,2101):
                    ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
                    res_val = tot_val[(year-1990)*4]

                    idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                    dfs[sim].loc[idx, opt] = ratio * res_val

        ## Deaths
        tot_val = 0
        for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
            tot_val += stores.get_variable(var)[0].vals + stores.get_variable(var)[1].vals

        for age in range(0,5):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, 'deaths'] = ratio * res_val

        tot_val = 0
        for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
            tot_val += stores.get_variable(var)[2].vals + stores.get_variable(var)[3].vals
        for age in range(5,15):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, 'deaths'] = ratio * res_val

        tot_val = 0
        for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
            tot_val += stores.get_variable(var)[4].vals + stores.get_variable(var)[5].vals
        for age in range(15,50):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, 'deaths'] = ratio * res_val

        tot_val = 0
        for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
            tot_val += stores.get_variable(var)[6].vals + stores.get_variable(var)[7].vals
        for age in range(50,70):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, 'deaths'] = ratio * res_val

        tot_val = 0
        for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
            tot_val += stores.get_variable(var)[8].vals + stores.get_variable(var)[9].vals
        for age in range(70,101):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, 'deaths'] = ratio * res_val

    final_df = dfs[1]
    for i in range(2,n_samples+1):
        final_df = pd.concat([final_df,dfs[i]])

    return final_df